In [1]:
from utils import build_url, save_json

API_KEY_FILE = 'api_key.txt'
REGION = 'euw1'
BASE_URL = 'api.riotgames.com'

QUEUE_TYPE = 'RANKED_SOLO_5x5'

# Ligas de las que extraer jugadores
PIPELINE = [
    {
        'url': '/lol/league/v4/challengerleagues/by-queue/{queue}',
        'name': 'CHALLENGER',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
        }
    },
    {
        'url': '/lol/league/v4/grandmasterleagues/by-queue/{queue}',
        'name': 'GRANDMASTER',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
        }
        
    },
    {
        'url': '/lol/league/v4/masterleagues/by-queue/{queue}',
        'name': 'MASTER',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
        }
    },
    {
        'url': '/lol/league/v4/entries/{queue}/{tier}/{division}',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
            'tier': 'DIAMOND',
            'division': 'I'
        },
        'page': 1
    },
    {
        'url': '/lol/league/v4/entries/{queue}/{tier}/{division}',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
            'tier': 'DIAMOND',
            'division': 'II',
        },
        'page': 1
    },
    {
        'url': '/lol/league/v4/entries/{queue}/{tier}/{division}',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
            'tier': 'DIAMOND',
            'division': 'III'
        },
        'page': 1
    },
    {
        'url': '/lol/league/v4/entries/{queue}/{tier}/{division}',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
            'tier': 'DIAMOND',
            'division': 'IV'
        },
        'page': 1
    }
]

with open(API_KEY_FILE) as f:
    api_key = f.read()

In [2]:
import requests, copy, time
import os, shutil
import dill as pickle

start = time.time()

execution_cache_dir = 'current_execution'
leagues_state_file = f'{execution_cache_dir}/remaining_leagues.pkl'
players_file = f'{execution_cache_dir}/players.pkl'

headers = {
  'X-Riot-Token': api_key
}

os.makedirs(execution_cache_dir, exist_ok=True)
MIN_DELAY = 20/100 # esperar 20ms entre peticiones
delay = MIN_DELAY

# Leer un fichero del estado de la ejecución si existe o copiar PIPELINE
# para empezar desde 0
if os.path.exists(leagues_state_file) and os.stat(leagues_state_file).st_size != 0:
    with open(leagues_state_file, 'rb') as f:
        leagues_current_execution = pickle.load(f)
else:
    leagues_current_execution = copy.deepcopy(PIPELINE[0:3]) # Para pruebas

if os.path.exists(players_file) and os.stat(players_file).st_size != 0:
    with open(leagues_state_file, 'rb') as f:
        players = pickle.load(f)
else:
    players = []

# Filtrar las fases que ya estén completas
leagues_current_execution = list(filter(lambda league: not league['completed'], leagues_current_execution))

for league in leagues_current_execution:
    # Para las ligas paginadas, pedir en bucle hasta que no se devuelvan jugadores
    # Si alguna petición falla, aumentar delay
    # Si va bien, disminuir el delay hasta el inicial
    # Al terninar cada fase o página, guardar el estado
    if 'page' in league:
        params = league['params']
        while True:
            url = build_url(REGION, BASE_URL, league['url'].format(**league['params']))
            response = requests.get(url, headers=headers, params={'page': league['page']})
            if response.status_code == 200:
                players_temp = response.json()
                if not players_temp:
                    break
                players.extend(players_temp)
                print(f"{params['tier']} {params['division']} ({league['page']}): len: {len(players_temp)} / total: {len(players)}")
                delay = max(delay/2, MIN_DELAY)
                league['page'] += 1
            else:
                retry_after = int(response.headers['Retry-After'])
                delay += retry_after
                print(f"{params['tier']} {params['division']} ({league['page']}): Petición fallida: {response.text}")
                print(f'Esperando {retry_after}s')
            with open(leagues_state_file, 'wb') as f:
                pickle.dump(leagues_current_execution, f)
            with open(players_file, 'wb') as f:
                pickle.dump(players, f)
            time.sleep(delay)
    else:
        url = build_url(REGION, BASE_URL, league['url'].format(**league['params']))
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            players_temp = response.json()['entries']
            players.extend(players_temp)
            print(f"{league['name']}: len: {len(players_temp)} / total: {len(players)}")
            delay = max(delay/2, MIN_DELAY)
        else:
            retry_after = int(response.headers['Retry-After'])
            delay += retry_after
            print(f"{params['name']}: Petición fallida: {response.text}")
            print(f'Esperando {retry_after}s')
        time.sleep(delay)
    league['completed'] = True
    with open(leagues_state_file, 'wb') as f:
        pickle.dump(leagues_current_execution, f)
    with open(players_file, 'wb') as f:
        pickle.dump(players, f)
        
end = time.time()
print(f'\nEl proceso ha tardado {end - start}s')

CHALLENGER: len: 300 / total: 300
GRANDMASTER: len: 700 / total: 1000
MASTER: len: 5148 / total: 6148

El proceso ha tardado 3.8817250728607178s


In [3]:
from utils import call_with_retry

# Guardar añadir el accountId para poder pedir partidas
summoners = {player['summonerId'] for player in players[0:100]} # Para pruebas limitar a 100 jugadores

accounts_file = f'{execution_cache_dir}/accounts.pkl'
summoners_with_account_file = f'{execution_cache_dir}/summoners_with_account.pkl'

if os.path.exists(accounts_file) and os.stat(accounts_file).st_size != 0:
    with open(accounts_file, 'rb') as f:
        accounts = pickle.load(f)
else:
    accounts = []
    
if os.path.exists(summoners_with_account_file) and os.stat(summoners_with_account_file).st_size != 0:
    with open(summoners_with_account_file, 'rb') as f:
        summoners_with_account = pickle.load(f)
else:
    summoners_with_account = set()

url = '/lol/summoner/v4/summoners/{}'

summoners = summoners - summoners_with_account

for summoner in summoners:
    full_url = build_url(REGION, BASE_URL, url).format(summoner)
    account = call_with_retry(full_url, headers)
    print(account['accountId'])
    accounts.append({
        'summoner': summoner,
        'account': account['accountId']
    })
    summoners_with_account.add(summoner)
    with open(accounts_file, 'wb') as f:
        pickle.dump(accounts, f)
    with open(summoners_with_account_file, 'wb') as f:
        pickle.dump(summoners_with_account, f)
    time.sleep(delay)

save_json(accounts, 'accounts.json')

qkVFtFB-BtPcEq8g9ivcISNtyqxMF-OS38l8H9y7BCyX1g
lOapMWCGTfdc5Z3HH5WvdAxpABVwZY9CLcD7vYoRq2JrXA
0NFzdAnsy3Mia_Pup0_0IpLEifeBmzCPKgjb9z3CxyANlRRbQnprDSOT
c3u0czKdYrEy0BFb_luVDeb_3fp_rVrG7EpqLRYhh15j-pn1i-M_jn4U
2dw_Tr_dJHAElbKJGjw01P7pC2RuRD1Tx3AFX36bnHAsiw
cpDa6rnHFIQP33_P-Thda6DBEFZQQCFYNIbcnClYbXKGrEc
gSfZtRohb2uZvXVOCxDugSQJy9_3tsoHEJ32ZTalphFisCM
89WPil9BuIOlgJ4VT4VwHzf_zooyqJKejbGSE-lT4HBYPic2K37M69gc
QaxMl33jIzl6N5Fju4dLMZrllVlwpYj5JEXVoQvKhoBAcg
5yk741Md35JtP9DSVwhB7chsjBx49rhHUjSv4gk_PRlPlQ
gwAcZCQktFyEa7O_LNzTDqTH1Se7Y0HxMRqU2dCDlRjIRQ
gF4p9FE8AwHa-uBCN1w-jOSykQiQthdvjVv_HeF6Qy4S-e4
ZH6iQbwCoird94TLvQYpFBpCln3U_avqsfOmZ_zslg44eHwkAFQeAAb1
zhHnYrxPOn9lgBVS51KIYQFS5bkue53H-cZ-QyDinDEE_A
zF84cUhSht6Qr6LBeEdxrooRaRb7crCac7wvxPp_r3Hdtrk
Ig2k7zPXPu28_DgMVOjTcwS8ZgDCCOxHFLE7_LEwq9QLB_0
k7UZvi0DhTgyU55Vk13ow2cLeiZVhPp2ZDMmqPhaPOxuvpInW7QCtMDh
MPdUp8h44Sliq9t_85i-d4YBiXT96ngIQ2zvJRwYwKlCliMGIR_y7sjI
eKWBDSxU0z3CGQZOz4SS9G5lZIEGcc7tZaoQ2VS7E1QRXNt4Dd11y55G
YPMktbHHwViQ1cOlHLV9DrEBRk910JfL

In [4]:
# Limpiar estados
shutil.rmtree(execution_cache_dir)